In [6]:



import spacy
#charger le modèle français simple.
nlp = spacy.load("fr_core_news_sm")
# traiter une phrase en utilisant le modèle
doc = nlp ( "Ceci est un texte que je traite avec Spacy" )
# Récupère le vecteur 'texte':
doc [ 4 ] .vector
# Obtenez le vecteur moyen pour la phrase entière (utile pour la classification des phrases, etc.)
doc.vector

array([-1.6647179 ,  1.0910058 , -0.72085845, -0.6931582 , -0.3491546 ,
       -0.9255777 ,  0.78966486,  1.5558457 ,  0.8096707 ,  0.04013576,
       -1.8671646 ,  0.49984232, -0.18524824, -0.552432  , -1.127025  ,
       -1.6343294 ,  0.40954483,  1.4451025 ,  0.49586126,  1.8007085 ,
       -1.4436576 , -0.32717168,  1.0135076 ,  1.6971691 ,  0.05813472,
       -2.4995332 , -0.13361377,  1.5594677 ,  0.4157923 ,  1.1484382 ,
        1.1631846 , -0.34370553,  0.06018662, -0.47846168, -1.2699313 ,
       -1.2447286 , -0.109083  , -0.4326124 , -1.3921127 ,  1.4142797 ,
       -1.0087551 ,  3.0353742 , -0.21573026, -0.34378222, -1.3377184 ,
        2.5178502 , -1.6089766 , -1.4561623 , -2.07325   ,  1.0594691 ,
       -0.65816844,  2.9645102 ,  0.96333194, -0.8749611 , -0.3860393 ,
       -0.45364666,  1.869631  ,  0.9347763 , -2.6552455 ,  0.5043463 ,
        0.55622864,  0.5570379 ,  0.24029694,  0.0859979 ,  3.195965  ,
       -1.366473  , -1.1995682 , -1.0624795 ,  0.4680774 ,  0.13

In [7]:
doc [ 4 ] .vector

array([-3.0265143 ,  2.7239542 , -5.0048556 ,  0.7257427 , -4.5300817 ,
       -0.25007123,  2.345086  ,  0.18997467,  0.5005402 , -0.37956768,
       -0.2795325 ,  4.0491633 ,  2.047511  ,  0.676478  ,  3.0325863 ,
       -1.577316  ,  5.7206454 ,  0.9426545 ,  0.14595929,  3.3483772 ,
       -2.1170335 , -1.231694  , -5.3810635 , -1.3933394 , -4.056142  ,
        0.21993887,  0.6674843 ,  2.2604933 , -0.4150899 ,  4.502517  ,
       -2.1294646 ,  5.4568644 , -2.1734705 ,  4.801753  , -3.9682686 ,
       -4.855811  , -0.08344412, -4.283447  , -4.328655  , -0.5942766 ,
       -1.9755992 ,  4.418608  , -1.6546574 , -0.44446576, -3.772001  ,
        2.2524805 , -0.65054476,  2.2739244 , -5.039021  ,  3.9841528 ,
       -0.6333385 ,  2.2404227 , -1.9977565 ,  2.8205366 , -2.4850612 ,
       -0.64855987,  2.7708547 ,  2.462861  , -4.6444483 ,  1.2306913 ,
        3.5711174 ,  0.5962813 , -0.00998145,  1.8171425 ,  5.6852083 ,
        0.9902173 ,  5.1536727 , -5.011366  ,  1.0806334 ,  0.31

### Gensim

#### Préparation des données 

In [14]:
import os
import sys
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
TEXT_DATA_DIR = 'C:/Users/lenovo/Downloads/20news/20_newsgroups'

In [15]:
# Newsgroups data is split between many files and folders.
# Directory stucture 20_newsgroup/<newsgroup label>/<post ID>

texts = []         # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []        # list of label ids
label_text = []    # list of label texts

# Go through each directory
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            # News groups posts are named as numbers, with no extensions.
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header in file (starts with two newlines.)
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)
                label_text.append(name)

print('Found %s texts.' % len(texts))

Found 19997 texts.


In [16]:
# Cleaning data - remove punctuation from every newsgroup text
sentences = []
# Go through each text in turn
for ii in range(len(texts)):
    sentences = [re.sub(pattern=r'[\!"#$%&\*+,-./:;<=>?@^_`()|~=]', 
                        repl='', 
                        string=x
                       ).strip().split(' ') for x in texts[ii].split('\n') 
                      if not x.endswith('writes:')]
    sentences = [x for x in sentences if x != ['']]
    texts[ii] = sentences

In [18]:
print(texts[6])

[['The', 'motto', 'originated', 'in', 'the', 'StarSpangled', 'Banner', '', 'Tell', 'me', 'that', 'this', 'has'], ['something', 'to', 'do', 'with', 'atheists'], ['The', 'motto', 'oncoins', 'originated', 'as', 'a', 'McCarthyite', 'smear', 'which', 'equated', 'atheism'], ['with', 'Communism', 'and', 'called', 'both', 'unamerican'], ['No', 'it', "didn't", '', 'The', 'motto', 'has', 'been', 'on', 'various', 'coins', 'since', 'the', 'Civil', 'War'], ['It', 'was', 'just', 'required', 'to', 'be', 'on', 'all', 'currency', 'in', 'the', "50's"], ['keith']]


In [19]:
# concatenate all sentences from all texts into a single list of sentences
all_sentences = []
for text in texts:
    all_sentences += text

In [20]:
# concatenate all sentences from all texts into a single list of sentences
all_sentences = []
for text in texts:
    all_sentences += text

In [21]:
# Phrase Detection
# Give some common terms that can be ignored in phrase detection
# For example, 'state_of_affairs' will be detected because 'of' is provided here: 
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(all_sentences, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)

# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])

In [22]:
model = Word2Vec(all_sentences, 
                 min_count=3,   # Ignore words that appear less than this
                 size=200,      # Dimensionality of word embeddings
                 workers=2,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=30)       # Number of epochs training over corpus

In [23]:
model.vector_size

200

In [28]:
model.most_similar("machine")

<ipython-input-28-c854c427bf40>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar("machine")


[('modem', 0.5224773287773132),
 ('system', 0.48703402280807495),
 ('hard_disk', 0.4633997678756714),
 ('mouse', 0.4599805474281311),
 ('computer', 0.4442451000213623),
 ('controller', 0.4432569742202759),
 ('client', 0.4394943118095398),
 ('mac', 0.43801236152648926),
 ('drive', 0.43596261739730835),
 ('workstation', 0.4348524212837219)]